In [1]:
import pandas as pd

In [44]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
# Загрузим файл с оперативными сводками
file_path = "contest/train_orig.xlsx"
df_train_orig = pd.read_excel(file_path, header=None)

# Посмотрим на первые строки
df_train_orig.columns = ["message"]
df_train_orig["label"] = 1  # 1 означает "оперативная сводка"
df_train_orig.head()

,message,label
0,Пахота зяби под мн тр\nПо Пу 26/488\nОтд 12 26...,1
1,Пахота зяби под мн тр\nПо Пу 26/514\nОтд 12 26...,1
2,Пахота зяби под сою\nПо Пу 29/1367\nОтд 16 29/...,1
3,12.10\nВнесение мин удобрений под оз пшеницу 2...,1
4,Пахота зяби под сою\nПо Пу 15/1382\nОтд 16 15/...,1


In [3]:
import random

negative_examples = [
    "Завтра трактор готов будет к 10 утра",
    "У нас кончилось топливо, нужно подвезти срочно",
    "Петрович, посмотри давление в шинах",
    "Обед с 12 до 13, потом продолжим работу",
    "Ключи от трактора оставил на вахте",
    "Нужно заказать запчасти на культиватор",
    "Аккумулятор сел, нужен новый",
    "Кто сегодня вечером остается на дежурство?",
    "Машина загрузилась удобрениями, едет к вам",
    "Какая завтра погода?",
    "Сделай перерыв на полчаса",
    "Тракторист заболел, нужен заменяющий",
    "С утра проверяйте масло",
    "Заправка техники сегодня будет позже",
    "Какие планы на день по посевной?",
    "Проверьте уровень охлаждающей жидкости",
    "Запчасть привезут только завтра утром",
    "Сломалась навеска, надо отремонтировать",
    "Сегодня больше не успеем, заканчиваем",
    "Кто последний работал на комбайне?",
    "Миша, проверь прикатывание посевов на 3-м участке, следы как будто поверхностные.",
    "Кто сегодня берет культиватор? У меня подкормка на подсолнечнике.",
    "Начали дискование, но трактор опять греется.",
    "На гуаре заметили сорняки, нужно внести гербицид.",
    "Пшеница товарная на поле 7 уже подошла, можно убирать.",
    "У кого ключ от склада? Фунгицид закончился.",
    "На овсе снова тля, нужна инсектицидная обработка.",
    "Чизель заглубляется слабо, может, дискование сначала сделать?",
    "Горох товарный дали на северный край, тракториста туда отправьте.",
    "Кто работает на семенной сое? Нужно уточнить норму сева.",
    "После предпосевной культивации трактор буксует, почва тяжёлая.",
    "Трактор с культиватором встал — цепь оборвалась, нужна помощь.",
    "Конопля взошла хорошо, прикатывание не требуется.",
    "Пахота на сорго завершена, переходим на свеклу.",
    "Рапс озимый нормально перезимовал, можно вносить удобрения.",
    "Люцерна уже в фазе бутонизации, подготовьте косилки.",
    "Пшеницу озимую товарную посмотрели, есть очаги ржавчины.",
    "Горох на зерно после чизлевания выглядит слабо, неравномерно взошёл.",
    "Кукуруза семенная — норма высева 28 тысяч, проверьте.",
    "Кто на боронование довсходовое? Ветер усилился — надо поторопиться.",
    "После уборки просо оставьте технику — будет подкормка.",
    "Сев овса завершили, прикатывание делать не будем.",
    "У кого бак под гербицид? Надо внести по многолетним злаковым.",
    "У нас проблемы с выравниванием на чистом пару — надо второй проход.",
    "На чумизе видно ожоги — возможно, ошибка с дозировкой удобрений.",
    "Семенной рапс плохо всходит, вызовите агронома.",
    "Сорго-суданковый гибрид идёт нормально, но сорняки прут.",
    "Кто на тракторе возле склада? Загрузи прикатчик.",
    "На многолетние травы прошлых лет вносим инсектицид — листоед пошёл.",
    "У кого смена после 16:00? Нужно закончить выравнивание зяби.",
    "Внесение минеральных удобрений задерживается — трактор не вышел.",
    "На подсолнечник товарный готовим агрегат для междурядной.",
    "Кто видел, куда выгрузили люцерну с последнего укоса?",
    "У кукурузы кормовой хорошая всхожесть, можно начинать междурядку.",
    "Вчера начали уборку на ячмене, вечером дождь испортил всё.",
    "Кто-то перепутал поле: вместо рапса начали сеять овёс.",
    "Говорят, у Коли отказали тормоза на прикатывателе — кто рядом?",
    "Фунгицидная обработка прошла, но пятна остались — нужна повторная.",
    "Соя товарная готова к уборке, нужна техника к утру.",
    "Составьте список, где уже была междурядная культивация.",
    "по отд отчитайтесь",
]

garbage_messages = [random.choice(negative_examples) for _ in range(100)]

df_negative = pd.DataFrame({"message": garbage_messages})
df_negative["label"] = 0

# Объединяем и продолжаем пайплайн
df_train_full = pd.concat([df_train_orig, df_negative], ignore_index=True)

In [4]:
# Импорт необходимых библиотек для пайплайна
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import FunctionTransformer

# Ключевые слова
KEYWORDS = ["/", "пу", "отд", "га", "день", "начала"]

def preprocess_text(text):
    text = text.lower()
    # Заменим кавычки, тире и лишние символы (кроме "/") на пробел
    text = re.sub(r"[\"\'“”«»„‟]", " ", text)          # кавычки → пробел
    text = re.sub(r"[-–—]", " ", text)                 # тире → пробел
    text = re.sub(r"[^\w\s/]", " ", text)              # прочее, кроме / → пробел
    text = re.sub(r"\s+", " ", text).strip()           # нормализация пробелов
    return text

In [5]:
# Применим предобработку
df_train_full["clean_text"] = df_train_full["message"].apply(preprocess_text)

In [6]:
df_train_full

,message,label,clean_text
0,Пахота зяби под мн тр\nПо Пу 26/488\nОтд 12 26...,1,пахота зяби под мн тр по пу 26/488 отд 12 26/2...
1,Пахота зяби под мн тр\nПо Пу 26/514\nОтд 12 26...,1,пахота зяби под мн тр по пу 26/514 отд 12 26/2...
2,Пахота зяби под сою\nПо Пу 29/1367\nОтд 16 29/...,1,пахота зяби под сою по пу 29/1367 отд 16 29/76...
3,12.10\nВнесение мин удобрений под оз пшеницу 2...,1,12 10 внесение мин удобрений под оз пшеницу 20...
4,Пахота зяби под сою\nПо Пу 15/1382\nОтд 16 15/...,1,пахота зяби под сою по пу 15/1382 отд 16 15/77...
...,...,...,...
201,Кто на тракторе возле склада? Загрузи прикатчик.,0,кто на тракторе возле склада загрузи прикатчик
202,Кто на боронование довсходовое? Ветер усилился...,0,кто на боронование довсходовое ветер усилился ...
203,"На гуаре заметили сорняки, нужно внести гербицид.",0,на гуаре заметили сорняки нужно внести гербицид
204,"Аккумулятор сел, нужен новый",0,аккумулятор сел нужен новый


In [7]:
# Трансформер: ключевые слова
class KeywordFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, keywords):
        self.keywords = keywords

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return np.array([[1 if kw in text else 0 for kw in self.keywords] for text in X])

# Трансформер: наличие даты
def extract_has_date_feature(X):
    pattern = re.compile(r"\b\d{1,2}[./-]\d{1,2}([./-]\d{2,4})?\b|\b\d{4}-\d{2}-\d{2}\b")
    return np.array([[1 if pattern.search(text) else 0] for text in X])

In [8]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    df_train_full["clean_text"], df_train_full["label"], test_size=0.2, random_state=42
)

# Построим пайплайн с дополнительной фичей "has_date"
pipeline_with_date = Pipeline([
    ("features", FeatureUnion([
        ("tfidf", TfidfVectorizer(ngram_range=(1,2))),
        ("keywords", KeywordFeatures(KEYWORDS)),
        ("has_date", FunctionTransformer(extract_has_date_feature, validate=False))
    ])),
    ("clf", LogisticRegression(max_iter=1000))
])

In [9]:
# Обучим и оценим
pipeline_with_date.fit(X_train, y_train)
y_pred_with_date = pipeline_with_date.predict(X_test)
report_with_date = classification_report(y_test, y_pred_with_date, output_dict=False)
print(report_with_date)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        22
           1       1.00      1.00      1.00        20

    accuracy                           1.00        42
   macro avg       1.00      1.00      1.00        42
weighted avg       1.00      1.00      1.00        42



In [10]:
pipeline_with_date

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('tfidf',
                                                 TfidfVectorizer(ngram_range=(1,
                                                                              2))),
                                                ('keywords',
                                                 KeywordFeatures(keywords=['/',
                                                                           'пу',
                                                                           'отд',
                                                                           'га',
                                                                           'день',
                                                                           'начала'])),
                                                ('has_date',
                                                 FunctionTransformer(func=<function extract_has_date_feature at 0x000001E019FF8400>))])),
                ('clf', LogisticRegression(max_iter=1000))])

In [11]:
# Функция применения обученного пайплайна к новым сообщениям
def apply_pipeline_to_messages(messages):
    original_messages = pd.Series(messages, name="original_message")
    clean_messages = original_messages.apply(preprocess_text)
    has_date_flags = original_messages.apply(lambda text: bool(re.search(r"\b\d{1,2}[./-]\d{1,2}([./-]\d{2,4})?\b|\b\d{4}-\d{2}-\d{2}\b", text)))

    predictions = pipeline_with_date.predict(clean_messages)

    result_df = pd.DataFrame({
        "original_message": original_messages,
        "clean_text": clean_messages,
        "has_date": has_date_flags,
        "is_operational_summary": predictions
    })

    return result_df



In [12]:
# Пример применения на трёх сообщениях
sample_new_messages = [
    "12.04.2025 Пахота зяби по ПУ 14/355",
    "Кто-нибудь сегодня делал предпосевную на кукурузе?",
    "Поставьте плуг к обеду на северный участок"
]
df_result = apply_pipeline_to_messages(sample_new_messages)
df_result

,original_message,clean_text,has_date,is_operational_summary
0,12.04.2025 Пахота зяби по ПУ 14/355,12 04 2025 пахота зяби по пу 14/355,True,1
1,Кто-нибудь сегодня делал предпосевную на кукур...,кто нибудь сегодня делал предпосевную на кукурузе,False,0
2,Поставьте плуг к обеду на северный участок,поставьте плуг к обеду на северный участок,False,0


In [13]:
df_result = apply_pipeline_to_messages(df_train_orig["message"])
df_result.head(10)

,original_message,clean_text,has_date,is_operational_summary
0,Пахота зяби под мн тр\nПо Пу 26/488\nОтд 12 26...,пахота зяби под мн тр по пу 26/488 отд 12 26/2...,False,1
1,Пахота зяби под мн тр\nПо Пу 26/514\nОтд 12 26...,пахота зяби под мн тр по пу 26/514 отд 12 26/2...,False,1
2,Пахота зяби под сою\nПо Пу 29/1367\nОтд 16 29/...,пахота зяби под сою по пу 29/1367 отд 16 29/76...,False,1
3,12.10\nВнесение мин удобрений под оз пшеницу 2...,12 10 внесение мин удобрений под оз пшеницу 20...,True,1
4,Пахота зяби под сою\nПо Пу 15/1382\nОтд 16 15/...,пахота зяби под сою по пу 15/1382 отд 16 15/77...,True,1
5,Пахота зяби под мн тр\nПо Пу 13/540\nОтд 12 13...,пахота зяби под мн тр по пу 13/540 отд 12 13/2...,False,1
6,15.10\nВнесение мин удобрений под оз пшеницу 2...,15 10 внесение мин удобрений под оз пшеницу 20...,True,1
7,15.10\nВнесение противозлакового гербецида на ...,15 10 внесение противозлакового гербецида на о...,True,1
8,Пахота зяби под мн тр\nПо Пу 26/566\nОтд 12 26...,пахота зяби под мн тр по пу 26/566 отд 12 26/2...,True,1
9,Пахота зяби под мн тр\nПо Пу 13/579\nОтд 12 13...,пахота зяби под мн тр по пу 13/579 отд 12 13/3...,False,1


In [23]:
import re
from datetime import datetime
import locale
# Устанавливаем русскую локаль
locale.setlocale(locale.LC_TIME, 'ru_RU.UTF-8')

'ru_RU.UTF-8'

In [24]:
from datetime import datetime
import dateparser

In [25]:
# Функция извлечения строки с датой
def extract_date_string(text):

    # Только форматы с точками, НЕ тире
    patterns = [
        r"\b\d{1,2}\.\d{1,2}\.\d{2,4}(г\.?)?\b",  # 28.03.2025г
        # r"\b\d{4}-\d{2}-\d{2}\b",              # отключено: 2024-04-12
        r"\b\d{1,2}\.\d{1,2}\b",                 # 12.04
        # r"\b\d{1,2}\s+\d{1,2}\b",                 # 12 04 (интерпретируем как день + номер месяца)
        r"\b\d{1,2}\s+[а-яА-Я]+\b"               # 12 марта
    ]
    for pattern in patterns:
        match = re.search(pattern, text)
        if match:
            return match.group(0)

    # Натуральные форматы (сегодня, вчера и т.п.)
    natural_keywords = ["вчера", "сегодня", "позавчера", "завтра"]
    for word in natural_keywords:
        if re.search(rf"\b{word}\b", text.lower()):
            return word

    return None

# Функция разбора даты и флага неполной даты
def parse_date_with_flag(date_str):
    if not date_str:
        return None, False
    current_year = datetime.now().year
    date_str = re.sub(r"(?<=\d)\s*[гГ]\.?\b", "", date_str).strip()

    # Полные даты с годом (4-значный или 2-значный)
    for fmt in ("%d.%m.%Y", "%d-%m-%Y", "%Y-%m-%d", "%d.%m.%y", "%d-%m-%y"):
        try:
            parsed = datetime.strptime(date_str, fmt).date()
            return parsed, False
        except ValueError:
            continue

    # Частичные даты без года → добавляем текущий год
    for fmt in ("%d.%m", "%d-%m", "%d %B"):
        try:
            dt = datetime.strptime(date_str, fmt)
            return dt.replace(year=current_year).date(), True
        except ValueError:
            continue

    # Сначала попытка через dateparser с явной базовой датой
    parsed = dateparser.parse(
        date_str,
        languages=['ru'],
        settings={
            'RELATIVE_BASE': datetime.now(),
            'PREFER_DAY_OF_MONTH': 'current',
            'PREFER_DATES_FROM': 'past',
            'RETURN_AS_TIMEZONE_AWARE': False,
            'DATE_ORDER': 'DMY'
        }
    )

    if parsed:
        is_partial = parsed.year == 1900
        if is_partial:
            parsed = parsed.replace(year=current_year)
        return parsed.date(), is_partial

    # Если dateparser не справился — возвращаем None
    return None, False

# Обновим функцию применения модели и извлечения даты
def apply_pipeline_with_date_and_flag(messages):
    original_messages = pd.Series(messages, name="original_message")
    clean_messages = original_messages.apply(preprocess_text)
    date_strings = original_messages.apply(extract_date_string)

    parsed_results = date_strings.apply(parse_date_with_flag)
    parsed_dates = parsed_results.apply(lambda x: x[0])
    is_partial_flags = parsed_results.apply(lambda x: x[1])

    has_date_flags = date_strings.notnull()
    predictions = pipeline_with_date.predict(clean_messages)

    result_df = pd.DataFrame({
        "original_message": original_messages,
        "clean_text": clean_messages,
        "date_string": date_strings,
        "parsed_date": parsed_dates,
        "is_partial_date": is_partial_flags,
        "has_date": has_date_flags,
        "is_operational_summary": predictions
    })

    return result_df

In [26]:
# Пример применения на трёх сообщениях
sample_new_messages = [
    "28.03.25г Пахота зяби по ПУ 14/355",
    "12 марта Пашем по отд 5",
    "вчера пахали по ПУ 15/100",
    "сегодня обработка ячменя",
    "28 03 Поставьте плуг на северный участок "
]
df_result = apply_pipeline_with_date_and_flag(sample_new_messages)
df_result

,original_message,clean_text,date_string,parsed_date,is_partial_date,has_date,is_operational_summary
0,28.03.25г Пахота зяби по ПУ 14/355,28 03 25г пахота зяби по пу 14/355,28.03.25г,2025-03-28,False,True,1
1,12 марта Пашем по отд 5,12 марта пашем по отд 5,12 марта,2025-03-12,False,True,0
2,вчера пахали по ПУ 15/100,вчера пахали по пу 15/100,вчера,2025-04-14,False,True,1
3,сегодня обработка ячменя,сегодня обработка ячменя,сегодня,2025-04-15,False,True,0
4,28 03 Поставьте плуг на северный участок,28 03 поставьте плуг на северный участок,03 Поставьте,None,False,True,0


In [27]:
df_result_with_flag = apply_pipeline_with_date_and_flag(df_train_orig["message"])
df_result_with_flag

,original_message,clean_text,date_string,parsed_date,is_partial_date,has_date,is_operational_summary
0,Пахота зяби под мн тр\nПо Пу 26/488\nОтд 12 26...,пахота зяби под мн тр по пу 26/488 отд 12 26/2...,None,None,False,False,1
1,Пахота зяби под мн тр\nПо Пу 26/514\nОтд 12 26...,пахота зяби под мн тр по пу 26/514 отд 12 26/2...,None,None,False,False,1
2,Пахота зяби под сою\nПо Пу 29/1367\nОтд 16 29/...,пахота зяби под сою по пу 29/1367 отд 16 29/76...,None,None,False,False,1
3,12.10\nВнесение мин удобрений под оз пшеницу 2...,12 10 внесение мин удобрений под оз пшеницу 20...,12.10,2025-10-12,True,True,1
4,Пахота зяби под сою\nПо Пу 15/1382\nОтд 16 15/...,пахота зяби под сою по пу 15/1382 отд 16 15/77...,47\n\nПредп,None,False,True,1
...,...,...,...,...,...,...,...
101,"Полевые работы АОР, 13.11.2024.\nПахота под ку...",полевые работы аор 13 11 2024 пахота под кукур...,13.11.2024,2024-11-13,False,True,1
102,14.11 Мир\nПахота зяби под кукурузу 57 га день...,14 11 мир пахота зяби под кукурузу 57 га день ...,14.11,2025-11-14,True,True,1
103,14.11.24.\nСП Коломейцево \n\nПахота под сах.с...,14 11 24 сп коломейцево пахота под сах св день...,14.11.24,2024-11-14,False,True,1
104,ТСК\nВспашка под кукурузу 70 га/ с нарастающим...,тск вспашка под кукурузу 70 га/ с нарастающим ...,70 га,None,False,True,1


In [28]:
df_result_with_flag[["clean_text"]].to_csv("clean_text_only.csv", index=False)
